In [ ]:
import pathlib
import sys
import string

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import autocrit
import autocrit_tools

sys.path.append("..")

import shared.format
import shared.tools
import shared.criticalpoints as cps

plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
pickle_dir = pathlib.Path("data") / "linear-results"

analytical_cp_df = pd.read_pickle(pickle_dir / "analytical_cp_df.pkl")
analytical_cp_df["morse_index"] = analytical_cp_df["hessian_spectrum"].apply(cps.compute_index)

In [ ]:
MR_success_df = pd.read_pickle(pickle_dir / ("newtonMR-success.pkl"))
MR_failure_df = pd.read_pickle(pickle_dir / ("newtonMR-failure.pkl"))

In [ ]:
damped_success_df = pd.read_pickle(pickle_dir / ("newtonTR-success.pkl"))
damped_failure_df = pd.read_pickle(pickle_dir / ("newtonTR-failure.pkl"))

In [ ]:
gnm_success_df = pd.read_pickle(pickle_dir / ("gnm-success.pkl"))
gnm_failure_df = pd.read_pickle(pickle_dir / ("gnm-failure.pkl"))

In [ ]:
success_dfs = [MR_success_df, damped_success_df, gnm_success_df]
failure_dfs = [MR_failure_df, damped_failure_df, gnm_failure_df]

In [ ]:
titles = ["Newton-MR", "Damped Newton", "BTLS-GNM"]

In [ ]:
def calculate_success_probability(success_df, failure_df):
    return len(success_df) / (len(success_df) + len(failure_df))

def calculate_success_sem(success_df, failure_df):
    p = calculate_success_probability(success_df, failure_df)
    return np.sqrt(p * (1 - p)) / np.sqrt(len(success_df) + len(failure_df))

In [ ]:
[print(title, ": ",
       calculate_success_probability(success_df, failure_df),
       "+/-", calculate_success_sem(success_df, failure_df))
      for title, success_df, failure_df in zip(titles, success_dfs, failure_dfs)];

In [ ]:
def compute_iteration_ct(df):
    return df["losses"].apply(len).sum()


def compute_iteration_mean(df):
    return compute_iteration_ct(df) / len(df)


def compute_iteration_std(df):
    return df["losses"].apply(len).std()


def compute_iteration_sem(df):
    return compute_iteration_std(df) / np.sqrt(len(df))

In [ ]:
[print(title, ":",
       round(compute_iteration_mean(success_df)),
       "+/-", round(compute_iteration_sem(success_df)))
      for title, success_df in zip(titles, success_dfs)];

In [ ]:
elapsed_times = [27 + 1/3, 77.75, 27]

In [ ]:
times_per_cp = [60 * time / len(cp_df) for time, cp_df in zip(elapsed_times, success_dfs)]
times_per_cp

In [ ]:
fig, axs = plt.subplots(figsize=(18, 12), nrows=2, ncols=3)

sgn_axs = axs[0]
loss_index_axs = axs[1]

## Squared Gradient Norms

[cps.make_trajectory_panel(success_df, failure_df, ax=sgn_ax)
for success_df, failure_df, sgn_ax in zip(success_dfs, failure_dfs, sgn_axs)]
[ax.set_ylabel("") for ax in sgn_axs[1:]]

## Loss-Index

[cps.make_loss_index_panel(success_df, analytical_cp_df, ax=ax)
 for success_df, ax in zip(success_dfs, loss_index_axs)]
[ax.set_ylim(loss_index_axs[0].get_ylim()) for ax in loss_index_axs]
[ax.set_ylabel("") for ax in loss_index_axs[1:]]

## Legend,Titles, Panels

axs[-1, -1].legend(loc="lower right");

[ax.set_title(title, size=shared.format.TITLESIZE)
    for title, ax in zip(titles, sgn_axs)]

[shared.tools.add_panel_label(letter, ax, pos=(-0.1, 1.1), size=shared.format.TITLESIZE)
 for letter, ax in zip(string.ascii_uppercase, axs.flatten())]

plt.tight_layout();
plt.savefig("perf-comparison.pdf")